<a href="https://colab.research.google.com/github/jacob-jones23/MAT-422/blob/main/ModuleC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Chapter 19: Root Finding**

Motivation: As the name suggests, the roots of a function are one of its most important properties. Finding the roots of functions is important in many engineering applications such as signal processing and optimization. For simple functions such as *f(x) = ax$^{2}$ + bx + c,* you may already be familiar with the "quadratic formula"
\begin{align}
x_r = \frac{-b ± \sqrt{b^2-4ac}}{2a}
\end{align}

which gives *x$_{r}$*, the two roots of *f* exactly. However for more complicated functions, the roots can rarley be computed using such explicit, or exact, means.

#**19.1: Root Finding Problem Statement**
The **root** or **zero** of a function, *f(x)*, is an *x$_{r}$* such that *f(x$_{r}$)* = 0. For functions such as *f(x) = x$^{2}$ -9*, the roots are clearly 3 and -3. However other functions like *f(x). cos(x) - x*, determining an **analytic**, or exact, solution for the roots of functions can be difficult. For this case it is useful to generate numerical approximations of the roots of *f* and understand the limitations in doing so.


**TRY IT** Using *fsolve* function from scipy to compute the root of f(x) = cos(x) - x near -2.

In [ ]:
import numpy as np
from scipy import optimize
f = lambda x: np.cos(x) - x
r = optimize.fsolve(f, -2)
print("r =", r)

# Verify the solution is a root
result = f(r)
print("result =", result)

r = [0.73908513]
result = [0.]


**TRY IT** The function f(x) = $\frac{1}{x}$ has no root. Use the fsolve function to try to compute the root of *f(x)* = $\frac{1}{x}$. Turn on the *full_output* to see what's going on.

In [ ]:
f = lambda x: 1/x

r, infodict, ier, mesg = optimize.fsolve(f, -2, full_output = True)
print("r =", r)

result = f(r)
print("result =", result)

print(mesg)

r = [-3.52047359e+83]
result = [-2.84052692e-84]
The number of calls to function has reached maxfev = 400.


We can see that, the value r we got is not a root, even though the f(r) is a very small number. Sunce we turned on the full_output, which have more information. A message will be returned if no solution is found, and we can see mesg details for the cause of failire - "The number of calls to function has reached maxfev = 400"

#**19.2: Tolerance**
In engineering and science, **error** is a deviation from an expected or computed value. **Tolerance** is the level of error that is acceptable for an engineering application. We say that a computer program has **converged** to a solution when it has found a solution with an error smaller than the tolerance. When computing roots numerically, or conduction any other kind of numerical analysis, it is important to establish both a metric for error and a tolerance that is suitable for a given engineering/science application.

For computing roots, we want x$_{r}$ such that f(x$_{r}$) is very close to 0. Therefore |f(x)| is a possible choice for the measure of error since the smaller it is, the likelier we are to a root.

#**19.3: Bisection Method**
The **intermediate value theorem** says that if f(x) is a continuous function between a and b, and sign(f(a)) != sign(f(b)), then there must be a c, such that a < c < b and f(c) = 0.

The **bisection method** uses the intermediate value theorem iteratively to find roots. Let f(x) be a continuous function, and a and b be real scalar values such that a < b. Assume without loss of generality, that f(a) > 0 and f(b) < 0. Then by the intermediate value theorem, there must be a root on the poen interval (a,b). Now let m = $\frac{b++a}{2}$, the midpoint between a and b. If f(m) = 0 or is close enough, then m is a root. If f(m) > 0, then m is an improvement on the left bound, a, and there is guranteed to be a root on the open interval (m, b). If f(m) < 0, then m is an improvement on the right bound, b, and there is guaranteed to be a root on the open interval (a, m).

**TRY IT** Program a function my_bisection(f, a, b, tol) that approximates a root r of f, bounded by a and b to withing |f($\frac{b+a}{2}$)| < tol

In [ ]:
import numpy as np

def my_bisection(f, a, b, tol):

  if np.sign(f(a)) == np.sign(f(b)):
    raise Exception(
        "The scalars a and b do not bound a root")

  m = (a+b)/2

  if np.abs(f(m)) < tol:
    # stopping condition, report m as root
    return m
  elif np.sign(f(a)) == np.sign(f(m)):
    # case where m is an improvement on a.
    # Make recursive call with a = m
    return my_bisection(f, m, b, tol)
  elif np.sign(f(b)) == np.sign(f(m)):
    # case where m is an improvement on b.
    # Make recursive call with b = m
    return my_bisection(f, a, m, tol)

**TRY IT** The $\sqrt{2}$ can be computed as the root of the function f(x) = x$^{2}$ - 2.  Starting at a = 0 and b =2, use my_bisection to approximate the $\sqrt{2}$ to a tolerance of |f(x)| < 0.1 and |f(x)| < 0.01. Verify that the results are close to a root by plugging the root back into the function.

In [ ]:
f = lambda x: x**2 - 2

r1 = my_bisection(f, 0, 2, 0.1)
print("r1 =", r1)
r01 = my_bisection(f, 0, 2, 0.01)
print("r01 =", r01)

print("f(r1) =", f(r1))
print("f(r01) =", f(r01))

r1 = 1.4375
r01 = 1.4140625
f(r1) = 0.06640625
f(r01) = -0.00042724609375


**TRY IT** See what will happen when a = 2 and b = 4 for the above.

In [ ]:
my_bisection(f, 2, 4, 0.1)

Exception: The scalars a and b do not bound a root

#**19.4: Newton-Raphson Method**
Let f(x) be a smooth and continuous function and x$_{r}$ be an unknown root of f(x). Now assume that x$_{0}$ is a guess for x$_{r}$. Unless x$_{0}$ is a very lucky guess, f(x$_{0}$) will not be a root. Given this scenario, we want to find an x$_{1}$ that is an improvement on x$_{0}$ (i.e., closer to x$_{r}$ than x$_{0}$). If we assume that x$_{0}$ is "close enough" to x$_{r}$, then we can improve upon it by taking the linear approximation of f(x) around x$_{0}$, which is a line, and finding the intersection of this line with the x-axis. Written out, the linear approximation of f(x) around x$_{0}$ is f(x) = f(x$_{0}$) + f'(x$_{0}$)(x-x$_{0}$). Using this approximation, we find x$_{1}$ such that f(x$_{1}$) = 0. Plugging these values into the linear approximation results in the equation
\begin{align}
0 = f(x_0) + f'(x_0)(x_1 - x_0),
\end{align}
which when solved for x$_{1}$ is x$_{1}$ = x$_{0}$ - $\frac{f(x_0)}{f'(x_0)}$.

Written generally, a **newton step** computes an improved guess, x$_{i}$, using a previous guess x$_{i-1}$, and is given by the equation
\begin{align}
x_i = x_{i-1} - \frac{g(x_{i-1})}{g'(x_{i-1})}
\end{align}

The **Newton-Raphson Method** of finding roots iterates Newton steps from x$_{0}$ until the error is less than the tolerance.

**TRY IT** Again, the $\sqrt{2}$ is the root of the function f(x) = x$^2$ - 2. Using $x_{0}$ = 1.4 as a starting point, use the previous equation to estimate $\sqrt{2}$.  Compare the approximation with the value computed by Python's sqrt function.
\begin{align}
x = 1.4 - \frac{1.4^2 - 2}{2(1.4)} = 1.4142857142857144
\end{align}

In [ ]:
import numpy as np

f = lambda x: x**2 - 2
f_prime = lambda x: 2*x
newton_raphson = 1.4 - (f(1.4))/(f_prime(1.4))

print("newton_raphson =", newton_raphson)
print("sqrt(2) =", np.sqrt(2))

newton_raphson = 1.4142857142857144
sqrt(2) = 1.4142135623730951


**TRY IT** Write a function my_newton(f, df, x0, tol), where the output is an estimation of the root of f, f is a function object f(x), df is a function object to f'(x), x0 is an inital guess, and tol is the error tolerance. The error measurement should be |f(x)|.

In [ ]:
def my_newton(f, df, x0, tol):
  #outupt an estimation of the root of f
  # using newton-raphson method
  if abs(f(x0)) < tol:
    return x0
  else:
    return my_newton(f, df, x0 - f(x0)/df(x0), tol)

**TRY IT** Use my_newton to compute $\sqrt{2}$ to within tolerance of 1e-6 starting at x0 = 1.5.

In [ ]:
estimate = my_newton(f, f_prime, 1.5, 1e-6)
print("estimate =", estimate)
print("sqrt(2) =", np.sqrt(2))

estimate = 1.4142135623746899
sqrt(2) = 1.4142135623730951


**TRY IT** Compute a single Newton step to get an improved approximation of the root of the function f(x) = x$^{3}$ + 3x$^{2}$ - 2x -5 and an initial guess, x$_{0}$ = 0.29.

In [ ]:
x0 = 0.29
x1 = x0-(x0**3 + 3*x0**2 - 2*x0 -5)/(3*x0**2 + 6*x0 - 2)
print("x1 =", x1)

x1 = -688.4516883116648


#**19.5: Root Finding in Python**
Python has the existing root-finding functions for us to use to make things easy. The function we will use to find the root is f_solve from the scipy.optimize.

The f_solve function takes in many arguments but the most important two is the function you want to find the root, and the initial guess.

**TRY IT** Compute the root of the function f(x) = x$^{3}$ - 100x$^{2}$ -x + 100 using *f_solve*.

In [ ]:
from scipy.optimize import fsolve

In [ ]:
f = lambda x: x**3 - 100*x**2 -x+100

fsolve(f, [2,80])

array([  1., 100.])